numpy=1.15.4
keras = 2.2.4
tensorFlow = 1.12.0 

In [11]:
import numpy as np
import tensorflow as tf
import pandas as pd
from keras.utils import np_utils
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

x_train = x_train.astype('float32')/255.
x_test = x_test.astype('float32')/255.

x_train = x_train.reshape((-1, 784))
x_test = x_test.reshape((-1, 784))

N, M = x_train.shape
O = 1024
P = 256
T = 10
batch_size = 100
learning_rate = 0.001
training_epochs = 10

inp = tf.placeholder(shape=(None, M), dtype=tf.float32)
out = tf.placeholder(shape=(None, T), dtype=tf.float32)

weights1 = tf.Variable(tf.truncated_normal([M, O], stddev=np.sqrt(1/M)))
bias1 = tf.Variable(tf.ones([O])/O)
layer1 = tf.nn.relu(tf.matmul(inp, weights1) + bias1)

weights2 = tf.Variable(tf.truncated_normal([O, P], stddev=np.sqrt(1/O)))
bias2 = tf.Variable(tf.ones([P])/P)
layer2 = tf.nn.relu(tf.matmul(layer1, weights2) + bias2)

weights_output = tf.Variable(tf.truncated_normal([P, T], stddev=np.sqrt(1/P)))
bias_output = tf.Variable(tf.ones([T])/T)
evidence = tf.matmul(layer2, weights_output) + bias_output
output = tf.nn.softmax(evidence)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=evidence, labels= out)
cross_entropy = tf.reduce_mean(cross_entropy)

train_step = tf.train.RMSPropOptimizer(learning_rate).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(out, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar("Cost", cross_entropy)
tf.summary.scalar("Accuracy", accuracy)
summary_op = tf.summary.merge_all()

with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(training_epochs):
            print("Epoch: ", epoch)
            batch_count = int(N/batch_size)
            for i in range(batch_count):
                start, finish = batch_size*i, batch_size*(i+1)
                batch_x, batch_y = x_train[start:finish, :], y_train[start:finish, :]
                d = {inp: batch_x, out: batch_y}
                _, summary = sess.run([train_step, summary_op], feed_dict=d)
        print("Accuracy", accuracy.eval(feed_dict={inp: x_test, out: y_test}))

Epoch:  0
Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9
Accuracy 0.977


In [6]:
import numpy as np
import tensorflow as tf

from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense

(x_train, y_train), (x_test, y_test) = mnist.load_data()

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

x_train = x_train.astype('float32')/255.
x_test = x_test.astype('float32')/255.

x_train = x_train.reshape((-1, 784))
x_test = x_test.reshape((-1, 784))

N, M = x_train.shape
O = 1024
P = 256
T = 10
batch_size = 100
learning_rate = 0.001
training_epochs = 10

model = Sequential()
model.add(Dense(P, activation='relu'))
model.add(Dense(T, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=training_epochs, batch_size=batch_size)
score = model.evaluate(x_test, y_test, batch_size=batch_size)
print(score)

Using TensorFlow backend.


Epoch 1/10
60000/60000 [==============================] - 15s 258us/step - loss: 0.2771 - acc: 0.9202
Epoch 2/10
60000/60000 [==============================] - 14s 237us/step - loss: 0.1213 - acc: 0.9651
Epoch 3/10
60000/60000 [==============================] - 13s 211us/step - loss: 0.0810 - acc: 0.9762
Epoch 4/10
60000/60000 [==============================] - 11s 186us/step - loss: 0.0615 - acc: 0.9814
Epoch 5/10
60000/60000 [==============================] - 11s 182us/step - loss: 0.0483 - acc: 0.9857
Epoch 6/10
60000/60000 [==============================] - 13s 216us/step - loss: 0.0386 - acc: 0.9886
Epoch 7/10
60000/60000 [==============================] - 15s 248us/step - loss: 0.0322 - acc: 0.9904
Epoch 8/10
60000/60000 [==============================] - 17s 281us/step - loss: 0.0250 - acc: 0.9931
Epoch 9/10
60000/60000 [==============================] - 22s 360us/step - loss: 0.0208 - acc: 0.9941
Epoch 10/10
10000/10000 [==============================] - 2s 241us/step
[0.075040